# Импорт библиотек

In [1]:
from sqlalchemy import create_engine, inspect
import psycopg2

# Соединение к базе данных

In [2]:
connection = 'postgresql://{}:{}@{}:{}/{}'.format(
    'postgres',
    'postgres',
    'postgres',
    '5432',
    'postgres')

engine = create_engine(connection)

In [3]:
# Вспомогательная функция.
def get_data(query, e=engine):
    return pd.read_sql(query, e)

Убедимся что никаких таблиц в базе нет.

In [4]:
inspect(engine).get_table_names()

[]

Создадим таблицы orders и users.

In [5]:
q = '''

DROP TABLE IF EXISTS orders;

CREATE TABLE orders (id int NOT NULL,
                     phone varchar(15),
                     amount int);

INSERT INTO orders
VALUES (2335, '89651753289', 20000),
       (2903, '9068941202', 15000),
       (3345, '+74953574500', 75000),
       (3782, '9651753289', 12000),
       (4774, '89256412531', 50000),
       (5684, '9159873256', 125000),
       (6017, '79159873256', 90000),
       (6337, '9068941202', 25000),
       (6560, '4953574500', 17500),
       (8412, '+79256412531', 55000);

SELECT *
FROM orders;

'''
get_data(q)

,id,phone,amount
0,2335,89651753289,20000
1,2903,9068941202,15000
2,3345,+74953574500,75000
3,3782,9651753289,12000
4,4774,89256412531,50000
5,5684,9159873256,125000
6,6017,79159873256,90000
7,6337,9068941202,25000
8,6560,4953574500,17500
9,8412,+79256412531,55000


In [6]:
q = '''

DROP TABLE IF EXISTS users;

CREATE TABLE users (phone varchar(15) NOT NULL,
                    surname varchar(25),
                    sex varchar(10));

INSERT INTO users
VALUES ('9651753289', 'Иванов', 'man'),
       ('9256412531', 'Васечкина', 'woman'),
       ('9159873256', 'Семенов', 'man'),
       ('9068941202', 'Петров', 'man'),
       ('4953574500', 'Зуева', 'woman');

SELECT *
FROM users;

'''
get_data(q)

,phone,surname,sex
0,9651753289,Иванов,man
1,9256412531,Васечкина,woman
2,9159873256,Семенов,man
3,9068941202,Петров,man
4,4953574500,Зуева,woman


Выполним запрос чтобы найти id с максимальной транзакцией для пользователя.

In [7]:
q = '''

WITH t AS
  (SELECT u.surname,
          o.id,
          o.amount,
          max(o.amount) OVER (PARTITION BY u.surname) max_amount
   FROM orders o
   LEFT JOIN users u ON u.phone = right(o.phone, 10)
   WHERE u.sex = 'man')
SELECT surname,
       id,
       max_amount
FROM t
WHERE amount = max_amount
ORDER BY max_amount DESC

'''
get_data(q)

,surname,id,max_amount
0,Семенов,5684,125000
1,Петров,6337,25000
2,Иванов,2335,20000
